In [2]:
'''
Reference:
Example script from Keras Github Repository keras/examples/lstm_text_generation.py
https://github.com/keras-team/keras

Same configuration as the vanilla LSTM char-RNN for Donna Summer.
Trained on song lyrics of both Gordon Lightfoot and Donna Summer (half from each dataset)
'''

from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

#path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#text = io.open(path, encoding='utf-8').read().lower()
import pandas as pd
lyrics_data = pd.read_csv('songdata.csv')

# Gordon Lightfoot
lyrics_data_gordon = lyrics_data[lyrics_data.artist == 'Gordon Lightfoot']['text']

lyrics_concat_gordon = u''
for i in lyrics_data_gordon.index:
    lyrics_concat_gordon += lyrics_data_gordon[i]
    
# Donna Summer

lyrics_data_donna = lyrics_data[lyrics_data.artist == 'Donna Summer']['text']

lyrics_concat_donna = u''
for i in lyrics_data_donna.index:
    lyrics_concat_donna += lyrics_data_donna[i]
    
lyrics_concat_gordon_donna = lyrics_concat_gordon[:len(lyrics_concat_gordon)/2] + lyrics_concat_donna[:len(lyrics_concat_donna)/2]
    
print("input chars length")
print(len(lyrics_concat_gordon_donna))
text = lyrics_concat_gordon_donna
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# train the model, output generated text after each iteration
for iteration in range(1, 60):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

Using TensorFlow backend.


input chars length
233642
corpus length: 233642
total chars: 70
nb sequences: 77868
Vectorization...
Build model...

--------------------------------------------------
Iteration 1
Epoch 1/1
77868/77868 [==============================] - 51s - loss: 2.0655    

----- diversity: 0.2
----- Generating with seed: "t me  
  
Ooh, just the man I hoped you'"
t me  
  
Ooh, just the man I hoped you're got a dand  
I can't can't go  
  
And I we we we teet the seall the see the stand  
  
And I wee's the get a dain  
  
Oh can't get  
  
The see where the sting the got a  
I want the want the get it all the sees  
And I want the stand  
I want the can't a my the stand  
  
I want the stand a don't all the seep  
The see the stand  
  
I want the stand  
  
The can't can't can't a want the get

----- diversity: 0.5
----- Generating with seed: "t me  
  
Ooh, just the man I hoped you'"
t me  
  
Ooh, just the man I hoped you're got a let you's nighte  
I wist feen  
  
The maghe  
I cas the reall 

And I love your Spanish moss me, bbes hofe  
She cravend in my sangir down the rough  
nu why atwor  
And thirty just love how  
Hey've moundersestery tonight in a wanter daon  
At fasters fun a dimes and dance that me  
And nong athing wrute it baby  
Her man go make a night I beling  
When like a warter endide  
And not me baby  
ure play  
Cansh yes me  
lick of my or I  
When you and ay for the world be to aw all wild it

----- diversity: 1.2
----- Generating with seed: "sses flow  
And I love your Spanish moss"
sses flow  
And I love your Spanish moss the belesind is is I passin, hamey long pind  
  
the srimack baby threec me atwoss dick of for the rsice a but  
To conly get out on the hard  
Fade up iust's rrome  
I won't know you known  
 up not talk love, fucts her murt ach us mt, call wherey light sig t, like tuld me could beled and fiel  
Couling wines how repumastip on pbise what plays boughts  
Gon't know unkind  
I don't miss olsed to

------------------------------------

We thin' love's tho

----- diversity: 1.2
----- Generating with seed: "r hand  
Sit down young stranger  
I do "
r hand  
Sit down young stranger  
I do oh I  
dowin' ful  
And at it head who toly give  
  
Wapkered elses in angold me  
Ofty you've you  
You know me to know bad  
Of the way it come  
Seemben und, make me might  
Time I can't rain that trued  
Side to let all thisteg  
Tulring ut I nearn to be , my understarts  
Oh the endiret I never seem regood  
Why keep on the long thin disine thround you  
Once from long to turn the were  
Get

--------------------------------------------------
Iteration 12
Epoch 1/1
77868/77868 [==============================] - 38s - loss: 1.1057    

----- diversity: 0.2
----- Generating with seed: "it melt away  
  
If you find it  
Then "
it melt away  
  
If you find it  
Then she see she see is the money.  
  
I got your love, all I ever think about  
It's to streep to get of the man his some  
  
I got your love me  
I got your love me  
I 

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


...

I love to love you baby  
I love to

----- diversity: 0.5
----- Generating with seed: " that country song  
We played for just "
 that country song  
We played for just yes stand, in the money.  
From a faw a little  
And the sangine  
I will be there in the change she callen  
  
I want to hear in the good  
There's no wondow  
And the song the sun  
The song the sunsind  
And the strends upon the mountayfthing she work  
  
She's a bore and the town is for you  
And oh then sweet shanged the wind  
So you're lear do  
It's a night it more no  
I be minving them

----- diversity: 1.0
----- Generating with seed: " that country song  
We played for just "
 that country song  
We played for just i tall that you see  
So duch in the weak of me ome night  
Never down ahdous alone  
Oh, what you'd like to co spring again  
Cause your ladied boy to kind  
Take steevers an  
All the san, a kid, I won' lin' woullber  
This is emsing sun  
I woold believe to n couldnior  
But I could never said

I would this back do  
If tried to sleep  
It man you still for the winds of the so  
Why it makes in the dark  
Bring down the reign  
  
She cause with the kinds home  
  
It

----- diversity: 1.0
----- Generating with seed: "e  
Lose yourself  
Let love be  
  
I w"
e  
Lose yourself  
Let love be  
  
I will be overe show I know I'm one  
We'll be waiting for a eye  
Now sone  
get a resing  
Where on homes of life foretim in cours no  
nops  
And the ath repun do and gonna body ta kide he's choins and dayn on the bood  
The rainbow and the ppoor ullow of truena  
On hy'll boldI'd as lake from I need  
It mind and way, this if you life it men  
I lay the bunders st?  
Don't vigat you trey south  


----- diversity: 1.2
----- Generating with seed: "e  
Lose yourself  
Let love be  
  
I w"
e  
Lose yourself  
Let love be  
  
I willer the wind from it's in ynerrvin  
migacting  
  
It's nomethebblle is we new  
you'll be wooh in no my my faillos  
Broken wonder before you seen  
You

Alboth a call along to feer  
And the finevers I want to let it all time  
Tollke is play  
The roas haven come to flowin' a again  
Oh, they know it's so full,  
It's taken in a bore  
Bekind of the contlah so, when the hills away  
s over  
Then I was sutting you say on the fortetty and one who can't glan that long love  
I'm mady

----- diversity: 1.2
----- Generating with seed: " where you belong  
  
Don't you underst"
 where you belong  
  
Don't you understang, he's got good around the drught I reitn to war that all on with mosen hole  
Lord boaching cloud, what I'm flyxin' a paid, the way  
Sweet shoflunb ell  
  
I'm honeverestifled give nodr?  
And no?, now all the grostoch baby  
Ooh baby, in a my, canarad new  
The geve abouy  
What all haw a amade he was all dream you soon get you know no camimin're  
It's there bund, freeth'r of  
  
I'm to u

--------------------------------------------------
Iteration 44
Epoch 1/1
77868/77868 [==============================] - 37s - los

And the paded now we can ead in stirren't it kind of all somehind ot dil you, on the moloy are mine  
stry up to strend to pands home  
She would you're leed in youde
dion in hi way  
You haved sofine, make me  
If you put me her love  
As when it for of she will hear i'Soin in you  
yang here my wind about alone  
That love that I 've doin' fityolut at strees  
Light inside my love stages and lone and

--------------------------------------------------
Iteration 51
Epoch 1/1
77868/77868 [==============================] - 36s - loss: 0.8441    

----- diversity: 0.2
----- Generating with seed: " I don't have to hear  
I'll be alright "
 I don't have to hear  
I'll be alright a surtly  
And oh now  
And I would mise  
I want to go, it back aftale  
And the stranger of a whoing  
All the light I've got a lonely gain  
And in the man I love you see  
It live that I was all have a sailly  
I want to let me down  
I want to go, home  
It's all I ever think about  
It's a long what I won't e

The law her,  when love  
Where I will courut 

--------------------------------------------------
Iteration 58
Epoch 1/1
77868/77868 [==============================] - 36s - loss: 0.8278    

----- diversity: 0.2
----- Generating with seed: "e days  
You're gonna miss your Donna  
"
e days  
You're gonna miss your Donna  
It's all I seellein'  
The spring affair, spring afazen's around the bells it out furly face  
We're goina talk away you feel a would could be a you  
I love to love to be  
  
I would be afraid  
  
This here I'm yesterday  
I would be alavid  
And the promise of a sutuman't be sold  
I would be afraid  
  
This here I'm yesterday  
I would be afraid  
  
Now that he should had enough  
I would b

----- diversity: 0.5
----- Generating with seed: "e days  
You're gonna miss your Donna  
"
e days  
You're gonna miss your Donna  
Stie to tray  
I want you to love you see  
Now they we never known't be alroigh  
It's the pace  
  
The mame let me be with a heart  
Is th

In [8]:
import pandas as pd
lyrics_data = pd.read_csv('songdata.csv')

# Gordon Lightfoot
lyrics_data_gordon = lyrics_data[lyrics_data.artist == 'Gordon Lightfoot']['text']

lyrics_concat_gordon = u''
for i in lyrics_data_gordon.index:
    lyrics_concat_gordon += lyrics_data_gordon[i]
    
# Donna Summer

lyrics_data_donna = lyrics_data[lyrics_data.artist == 'Donna Summer']['text']

lyrics_concat_donna = u''
for i in lyrics_data_donna.index:
    lyrics_concat_donna += lyrics_data_donna[i]
    
lyrics_concat_gordon_donna = lyrics_concat_gordon[:len(lyrics_concat_gordon)/2] + lyrics_concat_donna[:len(lyrics_concat_donna)/2]
print(len(lyrics_concat_gordon_donna))
#print lyrics_concat_donna

233642


In [3]:
# sentence 1

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Loneliness is always looking for a frien'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Loneliness is always looking for a frien"
Loneliness is always looking for a frienes in the sky  
  
Sing that I'll be alright it time  
I'm sumat at the love  
If there could feel it to pets a read has the sea  
And the every man the dice  
And the every stay  
I'm sure at sleeping  
The true love could be a but your mind  
I'm sure a praying sion  
And

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


 they're all the love you there  
I'm sure a but her stand, in the fairy tome  
  
So cloved and the the could as she walks, the waters for the mehirn  
  
It is deep to the bets a memont of the sky  
  
She we like find a realykdown the beght away  
Black lady, black lady  
I don't matter you could be a big forlsing side  
  
She we like that you and me  
I would be afraid of the sky  
  
She we'd was a mansid  
She was you tried to be down  
I want to go, it  
If it it to fine  
I want to hear away  
Breakdown, over  
  
Oh promises are all my love to me  
  
Oh me home  
  
Oh melody  
Sweet like a wind  
That I can

----- diversity: 0.5
----- Generating with seed: "Loneliness is always looking for a frien"
Loneliness is always looking for a friene  
  
She got all I ever think about  
I got your love you out there  
If you're looke for a while of the sky  
  
Well, have understand who mistake a wart  
I'm a promise of me  
  
If you tried to feet.  
I'll cllver in my mind it's be a

In [4]:
# sentence 2

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "We go hard or we go home \nWe gon' do thi"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "We go hard or we go home 
We gon' do thi"
We go hard or we go home 
We gon' do this here I can dream  
So you're the glack on 

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


the day  
I still aftern the way the tells are  
I got your love and the sea hole  
  
If you're looke that I do my way  
I got your love and the stranger who can walk love  
I want to go, it  
If it it to fine  
Something with the love  
I'm sure you there a lot make your beloding  
To the love be so could stay  
I was not becahot me  
Before the way the day  
If you will feel you  
I will be goind it's wrott and the next to to be up in the side  
I'm gonna let it ride  

Ooh baby, we're not to the one who can  
And the tells a mount survine  
With the love be so could stay  
I was not becah on the reason the reas  
  
She work that hapo left me  
I'll do anything that you walked  
  
The way the strength she would be holy  
I'll do anything I love you  
  
I will be goin' everyon the streets  
To the mess behind  
I wish the money dawn  
The 

----- diversity: 0.5
----- Generating with seed: "We go hard or we go home 
We gon' do thi"
We go hard or we go home 
We gon' do this everythi

In [5]:
# sentence 3

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "At first I was afriad, I was petrified \n"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "At first I was afriad, I was petrified 
"
At first I was afriad, I was petrified 
I'm honey worked the waters for you  
And you can do is tale  
I've got a lot mo's all that you 

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


see  
I'll do anything you back at all  
I'm after in the sun  
To sut the world feel and me  
  
Oh promises are all my love to me  
Black day in my mind  
Anybody steally  
I don't wanna work that hard  
I do anything for love  
And the every the black on the day  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love y

----- diversity: 0.5
----- Generating with seed: "At first I was afriad, I was petrified 
"
At first I was afriad, I was petrified 
I'm hurt me fieling  
He tried in the mountains  
We we've fee

In [6]:
# sentence 4

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Young Money, Nicki Minaj, Justin  \n  \nSh"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Young Money, Nicki Minaj, Justin  
  
Sh"
Young Money, Nicki Minaj, Justin  
  
She's a friend to me  
And they with the money.  
So you better beware  
  
An

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


d in my love love  
It's all I ever think about  
I got your love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to l

----- diversity: 0.5
----- Generating with seed: "Young Money, Nicki Minaj, Justin  
  
Sh"
Young Money, Nicki Minaj, Justin  
  
She's alive in the mountains  
Sometime I'd 

In [7]:
# sentence 5

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Life is to be lived on the positive tip "
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Life is to be lived on the positive tip "
Life is to be lived on the positive tip  
  
Oh, nowhing the world was a mind  
  
It is dreaming on the rain  
I don't wanna work th

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


at hard  
I do anything that I do  
It's all I ever think about  
I got your love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I lo

----- diversity: 0.5
----- Generating with seed: "Life is to be lived on the positive tip "
Life is to be lived on the positive tip  
This head on the place the place distance  
  
Take my wa

In [8]:
# sentence 6

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Another night I'm waiting for you  \nWait"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Another night I'm waiting for you  
Wait"
Another night I'm waiting for you  
Waiting alone, they're the one that I'll breakion  
Stop season on the rain  
I don't wanna work that hard  
I do anything 

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


you  
I got your love and the stranger who ste work of the sun  
The soulde world  
I want to go, it's play  
  
I'm all the love and the stranger who can  
The soulde final  
And in my life in me  
  
I got your love at all  
I'm sure at the place to be awon  
All than I can do  
And the stear all the love you there always sead  
I'll do anything you back at all  
I this sains when I was a mind  
  
The winder that should I man in the sun  
There's a sea up the mornin'  
I'm gonna let it ride  

We all us I saw two sun  
The sweeter dance  
The stranger  
She's meen don't seemenderstangs  
  
To the soulde free  
I'll stranger in the sun  
Thtic baby on the rain  
I don't wanna work that hard  
  
She's a friften to the border bore  
I was day...blase  
  
She was aroun

----- diversity: 0.5
----- Generating with seed: "Another night I'm waiting for you  
Wait"
Another night I'm waiting for you  
Waitted the mornin' around  
  
I can't get to surver to the blacksence  
We're gonna tak

In [9]:
# sentence 7

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Back track, listen, stop that  \nBefore e'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Back track, listen, stop that  
Before e"
Back track, listen, stop that  
Before eables and the streets  
To the world we can  
Somether on the love with a way the way  
I would be afraid me  
  
Spirt me  
And if you're been the co

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


uld as all  
And when I was is to pat still  
I want to let is got  
There's a mooke my mind and the stranger of the sun  
The soulde gone  
I got your love  
It's all that I do  
It's all I ever think about  
I got your love and the strength seasing  
The trestle up the dicior  
If the stranger who my way  
Sweet love  
I'll do anything  
I got your love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to love you baby  
I love to

----- diversity: 0.5
----- Generating with seed: "Back track, listen, stop that  
Before e"
Back track, listen, stop that  
Before elaide  
I'm sure we can  
Oh hard for the money.  
So you better treat her loug  
I want to go trach run  
Like the 

In [10]:
# sentence 8

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'How can a non-musician discuss the futur'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "How can a non-musician discuss the futur"
How can a non-musician discuss the future  
I want to go  
Ooh blow me strong  
Sing that surve of my heart  
  
The trest me  
  
Spent sweet should I feel you the sing of the sun  
The sweeter dance  
  
She we like a will and the new fill  
And it's there a mollowhones  
She's a friend  
  
The trest freedown the way  
I would be afraid out of the sky  
  
I was all unter that he truer  
I'll there all I'm after if you 

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


tried  
  
The tells a moll over him me  
  
I'm gonna be the me

Oh, they can tell and the stranger who can walk love  
And the everywhing the way it's there a hold him  
Spirt a man like a wall  
  
We'd just a look along  
  
I'm gonna beat me a make you feel you and me  
If you will feel you feel your mind  
I'm sure at the dark  
And the love me  
And the stranger who can time  
There's a morort and the strength are cless fairy  
Behind I was days in the sun  
Thting for the sky  
  
Sing that she don't 

----- diversity: 0.5
----- Generating with seed: "How can a non-musician discuss the futur"
How can a non-musician discuss the future  
I got your love at the dark  
And the the grous and tend  
She's meen if I want to get a womach the bell  
I'm lound and you got all the love again  
The tells a mount through in the blue  
The tells a mountains  
Standic sing that whole remember  
  
She cause who you belong for a while  
Spring affair, spring affair, spring affair, spring affai

In [11]:
# sentence 9

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = 'Hi out there. This is Shaun Goldberg and'
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Hi out there. This is Shaun Goldberg and"
Hi out there. This is Shaun Goldberg and the end  
Why the memonceranny neet  
I'm gonna waiting it's away  
I want 

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


to go trach runs...................on't ddundets about  
I got your love at all  
I want to go, it's the green  
In the moliss and the the strength she was a ming the bright in the sky  
  
She was around  
I'm sumber day  
I got it  
I'm gonna wno we're gonna beace  
But I can't for someone they was a mill on the rain  
I don't wanna work that hard  
I do anything that you see  
I'm sumen the broke you  
I got your love and the stranger who can't who the sea hill sing you there was a mind  
  
The trest me  
  
Spear, pand heart of me  
  
Oh promises are all my love to me  
  
Black daycertant so long to be better out to find  
I will be goin't back dow again  
We've got a lot mo's all that I do  
It's all I ever think about  
I got your love to love you baby  
I love to love you baby  
I love to love you baby 

----- diversity: 0.5
----- Generating with seed: "Hi out there. This is Shaun Goldberg and"
Hi out there. This is Shaun Goldberg and be alone  
See you will feel good  
I'm g

In [12]:
# sentence 10

for diversity in [0.2, 0.5, 1.0, 1.2]:
    print()
    print('----- diversity:', diversity)

    generated = ''
    #sentence = text[start_index: start_index + maxlen]
    sentence = "Whose eyes in my behind  \nI don't recogn"
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(900):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


----- diversity: 0.2
----- Generating with seed: "Whose eyes in my behind  
I don't recogn"
Whose eyes in my behind  
I don't recogno bres you can't to be afraid  
  
They keep to the pacss shaic  
To the leave of your mem.  
To take a wall  
I was day it's all the thou

/opt/conda/lib/python2.7/site-packages/ipykernel_launcher.py:86: RuntimeWarning: divide by zero encountered in log


gh  
I'll do anything  
I got your love to be aloned  
I want to go  
  
Ooh baby, neware cliptive, the rein the dark  
And in my way  
And the mountains through she dream  
So strange  
If it he still all the love behind  
Someone just the soulde fifted to with her mind higher stand,  
I'm gonna let it ride  

I got your loving of the sky  
  
She we'd was a manside  
And the money.  
So you're up tell has the rest  
If the sousin' and dance  
  
She we'd was a mansid of the sun  
The spon is pring affair  
And I'm hurt all the love you  
I got your love and the the stranger of the sun  
The spoins as so coos  
  
There's a mooke my morning stranding in the sun  
The soulde find a way  
She's more like you  
I got your love to love you  
You know I'm 

----- diversity: 0.5
----- Generating with seed: "Whose eyes in my behind  
I don't recogn"
Whose eyes in my behind  
I don't recognorely to the better beware  
  
Take you feel love  
I'll something to sweet the coos  
The world gone a